This notebook is forked from https://www.kaggle.com/david1013/halite-game-scraper

# Set your team name here

In [ ]:
team_name = "shirakia+digimagi" # EDIT HERE

# Codes

In [ ]:
import pandas as pd
import numpy as np
# import os
import requests
# import json
from zipfile import ZipFile
import datetime
import time
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [30, 20]

In [ ]:
base_url = "https://www.kaggle.com/requests/EpisodeService/"
get_url = base_url + "GetEpisodeReplay"
list_url = base_url + "ListEpisodes"

r = requests.post(list_url, json = {"teamId":  5151726})
rj = r.json()
teams_df = pd.DataFrame(rj['result']['teams'])

In [ ]:
def getTeamEpisodes(team_id):
    # request
    r = requests.post(list_url, json = {"teamId":  int(team_id)})
    rj = r.json()

    # update teams list
    global teams_df
    teams_df_new = pd.DataFrame(rj['result']['teams'])
    
    if len(teams_df.columns) == len(teams_df_new.columns) and (teams_df.columns == teams_df_new.columns).all():
        teams_df = pd.concat( (teams_df, teams_df_new.loc[[c for c in teams_df_new.index if c not in teams_df.index]] ) )
        teams_df.sort_values('publicLeaderboardRank', inplace = True)
#         print('{} teams on file'.format(len(teams_df)))
    else:
        print('teams dataframe did not match')
    
    # make df
    team_episodes = pd.DataFrame(rj['result']['episodes'])
    team_episodes['avg_score'] = -1;
    
    for i in range(len(team_episodes)):
        agents = team_episodes['agents'].loc[i]
        agent_scores = [a['updatedScore'] for a in agents if a['updatedScore'] is not None]
        team_episodes.loc[i, 'submissionId'] = [a['submissionId'] for a in agents if a['submission']['teamId'] == team_id][0]
        team_episodes.loc[i, 'updatedScore'] = [a['updatedScore'] for a in agents if a['submission']['teamId'] == team_id][0]
        
        if len(agent_scores) > 0:
            team_episodes.loc[i, 'avg_score'] = np.mean(agent_scores)

    for sub_id in team_episodes['submissionId'].unique():
        sub_rows = team_episodes[ team_episodes['submissionId'] == sub_id ]
        max_time = max( [r['seconds'] for r in sub_rows['endTime']] )
        final_score = max( [r['updatedScore'] for r_idx, (r_index, r) in enumerate(sub_rows.iterrows())
                                if r['endTime']['seconds'] == max_time] )

        team_episodes.loc[sub_rows.index, 'final_score'] = final_score
        
    team_episodes.sort_values('avg_score', ascending = False, inplace=True)
    return rj, team_episodes

In [ ]:
team_id = int(teams_df[teams_df['teamName'] == team_name]['id'])
_rj, df = getTeamEpisodes(team_id)
df = df.sort_values(by=['id'])

In [ ]:
limit = 5000
df_limited = df.sort_values(by=['submissionId','id']).tail(limit).sort_values(by=['id'])
for key, grp in df_limited.reset_index().groupby('submissionId'):
    plt.plot(grp['updatedScore'],label=key)
plt.ylim(800, 1200)
xmax = len(df_limited)
plt.hlines(y=1000, xmin=0, xmax=xmax)
plt.hlines(y=1100, xmin=0, xmax=xmax)
plt.hlines(y=1200, xmin=0, xmax=xmax)
plt.show()